# Portfolio Strategy Backtesting
In this notebook we backtest the portfolio optimization strategy presented in example.ipynb. We will:
1. Select a market index (a portfolio) to construct a "sub-portfolio" from (e.g. the S&P 500 index).
2. Select a time frame and frequency over which we will obtain historical data from (e.g. 2010 to 2015, monthly).
3. Form the Market Correlation Graph (MCG) for the market index and solve the MWIS problem using TitanQ.
4. Assign portfolio weights to the outputted MWIS problem solution to form the final portfolio.
5. Evaluate the performance of the portfolio against the original market index over a future time frame (e.g. 2016 to 2017).

In [1]:
# Copyright (c) 2024, InfinityQ Technology Inc.

import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import logging
import warnings
from utils import *
from titanq import Model, Vtype, Target, S3Storage

# Filter out FutureWarning messages due to yfinance using a deprecated pandas keyword
warnings.simplefilter(action='ignore', category=FutureWarning)

## Setting Credentials
The user should configure their TitanQ API key here.

In [2]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API key here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Fetching Historical Data

In [ ]:
# Read in tickers and weights from the instance file
instance = "S&P 500"
tickers, weights = read_instance(f"instances/{instance}")

# Suggested: Set start_date and end_date to cover the entire time period you want to experiment on, so that you
# only have to pull data from Yahoo Finance once (in this cell)
# Recommended range is 2010-01-01 to 2023-12-31 for the S&P 500 instance
start_date = "2010-01-01"
end_date = "2023-12-31"

# Period of returns. 'M' is monthly
period = 'M'

# Get daily closing prices
close_prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close'][tickers]

# Prune delisted stocks
close_prices = close_prices.dropna(axis=1, how='all')

# Number of stocks in consideration (number of vertices in the graph)
size = len(close_prices.columns)

# Get periodic returns
daily_returns = get_stock_daily_returns(close_prices)
periodic_returns = daily_to_periodic_returns(daily_returns, period)

# Get market index close prices. ^GSPC is the ticker/stock symbol for the S&P 500
index_close_prices = yf.download(['^GSPC'], start=start_date, end=end_date, progress=False, auto_adjust=True)['Close']

## Setting Up The Problem
Here we form the Market Correlation Graph (MCG) by defining the adjacency matrix and node weights.



In [4]:
# Time frames for "training" the portfolio and for evaluating its performance
model_start_date = "2021-01-01"
model_end_date = "2021-12-31"
evaluation_start_date = "2022-01-01"
evaluation_end_date = "2022-12-31"

# Get close prices and returns for the training time period
close_prices_training = close_prices.loc[model_start_date:model_end_date]
returns = periodic_returns.loc[model_start_date:model_end_date]

# Compute correlation matrix and adjacency matrix for the Market Correlation Graph (MCG)
corr_matrix = get_stock_corr_matrix(returns)
theta = 0.45
J_matrix = corr_to_J_matrix(corr_matrix, theta)

# Compute (geometric) mean returns
mean_returns = get_stock_mean_returns(returns)

# Compute standard deviations of returns
stds = get_stock_stds(returns)

# Node weights in MCG
weights = np.array(mean_returns/stds, dtype=np.float32)

# Formulating The MWIS Problem And Solving On TitanQ

In [5]:
# Hyperparameters to tune
A = 1
B = 1

#############
# TitanQ SDK
#############
model = Model(
    api_key=TITANQ_DEV_API_KEY,     
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)

x = model.add_variable_vector('x', size, Vtype.BINARY)

# Setting Objective Using Expression
expr = A*0.5*np.dot(x.T, J_matrix.dot(x))-B*weights.dot(x)
model.set_objective_expression(expr)

In [6]:
# TitanQ Solver Hyperparameters
coupling_mult = 0.03
timeout_in_secs = 5
num_chains = 32
num_engines = 16
Tmin = 0.05
Tmax = 1
beta = (1/np.linspace(Tmin, Tmax, num_chains, dtype=np.float32)).tolist()

response = model.optimize(beta=beta, coupling_mult=coupling_mult, timeout_in_secs=timeout_in_secs, num_chains=num_chains, num_engines=num_engines)

## Printing The Results
In this section we print the results of the TitanQ solve, and map the solution to corresponding stock symbols.

In [ ]:
# Keep track of the index and the weight of the best solution
best_idx = -1
best_weight = 0

for idx, solution in enumerate(response.x):
    solution_weight = weights.dot(solution)

    if best_idx == -1 or solution_weight > best_weight:
        best_idx = idx
        best_weight = solution_weight
    
        
# Print the results of the best valid solution
print("\n-------- BEST VALID SOLUTION --------")

if best_idx == -1:
    print("None of the engines returned valid solutions!")
    print("Try adjusting the hyperparameters further to yield some valid solutions.")
else:
    portfolio_stocks = [tickers[i] for i in np.nonzero(response.x[best_idx])[0]]
    print(f"--- Engine {best_idx + 1} ---")
    print("stocks:", portfolio_stocks)
    print("weight:", best_weight)
    print("energy:", response.computation_metrics().get('solutions_objective_value')[best_idx])

# Portfolio Performance
In this section we form an "MWIS portfolio" with the MWIS solution we obtained above, and we evaluate its performance over a future time period.

In [ ]:
close_prices_evaluation = close_prices.loc[evaluation_start_date:evaluation_end_date]

# Portfolio with weights proportional to mean return/risk (same as node weights in the MCG for the MWIS problem)
portfolio_prices = close_prices_evaluation[portfolio_stocks[0]] * weights[tickers.index(portfolio_stocks[0])]
portfolio_weights = [weights[tickers.index(portfolio_stocks[0])]]

for i in range(1, len(portfolio_stocks)):
    stock_weight = weights[tickers.index(portfolio_stocks[i])]
    portfolio_prices += close_prices_evaluation[portfolio_stocks[i]] * stock_weight
    portfolio_weights.append(stock_weight)
    
portfolio_prices /= np.sum(portfolio_weights)

# Print portfolio stocks and weights
print("---- MWIS Portfolio ----")
portfolio_df = pd.DataFrame({"Ticker": portfolio_stocks, "Weight": portfolio_weights/np.sum(portfolio_weights)})
portfolio_df = portfolio_df.sort_values("Weight", ascending=False)
print(portfolio_df.to_string(index=False))

# Daily closing prices of the index
index_close_prices_evaluation = index_close_prices.loc[evaluation_start_date:evaluation_end_date]

# Dataframe with prices of the MWIS portfolio and the index
comparison_prices = pd.concat({"MWIS Portfolio": portfolio_prices, "S&P 500": index_close_prices_evaluation}, axis=1)
comparison_prices.fillna(method='ffill')
comparison_returns = comparison_prices.ffill().pct_change()
comparison_cumprod = comparison_returns.add(1).cumprod().sub(1)*100

# Plot cumulative returns of both the MWIS portfolio and the market index
fig = px.line(
    comparison_cumprod,
    x=comparison_cumprod.index,
    y=comparison_cumprod.columns,
    title="MWIS vs. S&P 500 Performance")

fig.update_layout(
    legend_title="Portfolio",
    width=1000,
    height=500
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cumulative Return (%)")

fig.show()

In [ ]:
# Compare the mean return, risk, and Sharpe ratio of both portfolios
comparison_periodic_returns = daily_to_periodic_returns(comparison_returns, 'M')

print("---- Mean Return (%) ----")
comparison_means = get_stock_mean_returns(comparison_periodic_returns)
print(comparison_means.to_string())

print("\n---- Risk (Std) (%) ----")
comparison_stds = get_stock_stds(comparison_periodic_returns)
print(comparison_stds.to_string())

print("\n---- Sharpe Ratio ----")
# Historical 1 month US treasury bill rates from 2010 to 2023
rf_rates = {2010: 0.15, 2011: 0.06, 2012: 0.04, 2013: 0.04, 2014: 0.03, 2015: 0.02, 2016: 0.18, 2017: 0.74, 2018: 1.62,
            2019: 2.43, 2020: 0.04, 2021: 0.01, 2022: 0.15, 2023: 4.70}
evaluation_start_year = int(evaluation_start_date[:4])
risk_free = rf_rates.get(evaluation_start_year)/12
comparison_sharpe_ratios = (comparison_means - risk_free)/comparison_stds
print(comparison_sharpe_ratios.to_string())